# GEOG310, Lab 8, Working with large datasets - calculationg ENSO 

This notebook introduces working with large climate datasets. 

Logging into see-trx4001
Step 1: open a user account using powershell on windows or linux/mac terminal
Step 2: secure shell into see-trx4001 linux computer ssh usr123@see-trx4001.canterbury.ac.nz then hit enter
Step 3: use your internet browser to log into the JupyterLab server https://see-trx4001.canterbury.ac.nz:9552/


ENSO can be defined using several parameters:

* Sea Lelvel Pressure diffrerence (SOI) The negative phase of the SOI represents below-normal air pressure at Tahiti and above-normal air pressure at Darwin. 
https://www.ncei.noaa.gov/access/monitoring/enso/soi

* See Surface temperature anomaly
El Niño (La Niña) is a phenomenon in the equatorial Pacific Ocean characterized by a five consecutive 3-month running mean of sea surface temperature (SST) anomalies in the Niño 3.4 region that is above (below) the threshold of +0.5°C (-0.5°C). This standard of measure is known as the Oceanic Niño Index (ONI).

* SST Principal Component Analysis 
    https://kls2177.github.io/Climate-and-Geophysical-Data-Analysis/chapters/Week7/Intro_to_PCA.html
    

<b> There are 11 Tasks in this notebook.</b>

<b> Section 1 - Calculate ONI index -tasks 1-3</b>

<b> Section 2 - CLimate in NZ - tasks 4 - 5  </b>

<b> Section 3 - NZ and ENSO -  tasks 6 - 11 </b>
    


### Section 1.  Calculate ONI index

SST dataset: ERSST version 5
https://psl.noaa.gov/data/gridded/data.noaa.ersst.v5.html

Monthly values, 1854 - present


How the dataset has been dowloaded:

`mkdir ERSST`

`cd ERSST`

`wget -r -nd -R "index.html*" https://www.ncei.noaa.gov/pub/data/cmb/ersst/v5/netcdf/ .`

 The dataset comes in yearly files, but has been merged into one file for your convinience

`cdo mergetime ersst.*.nc ersst_all.nc`


#### Section 1.1 - plot the values from the file, define the ONI region

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
from scipy.signal import detrend
import pandas as pd
import warnings

#library to handle time variable in the SST file
import cftime as cftime
warnings.filterwarnings('ignore')

In [ ]:
#Load the dataset
sst = xr.open_dataset("ERSST_full_record.nc")

#as this is a single level dataset, we can remove that dimention
sst = xr.open_dataset("ERSST_full_record.nc").squeeze("lev").drop("lev")

<b>Task 1: Inverstigate 2 variables in the dataset</b>

<b>What are their names:</b>

<b>Time period:</b>

<b>Horizontal resolution:</b>

In [ ]:
# define Nino3.4 region boundaries
lon1 = 360-170; lon2 = 360-120
lat1 = -5; lat2 = 5
# also the other NINO regions
# Nino1.2
lon1_12 = 360-90; lon2_12 = 360-80
lat1_12 = -10; lat2_12 = 0
# Nino3
lon1_3 = 360-150; lon2_3 = 360-90
lat1_3 = -5; lat2_3 = 5
# Nino4
lon1_4 = 160; lon2_4 = 360-150
lat1_4 = -5; lat2_4 = 5

In [ ]:
# define projection of SST data: lonlat grid -> "PlateCarree()"
data_proj = ccrs.PlateCarree()
# define projection for maps
proj = ccrs.PlateCarree(central_longitude=180)

# create figure and axis with map projection
fig, ax = plt.subplots(1,1,subplot_kw={'projection':proj})
# draw some coastlines
ax.coastlines();

In [ ]:
# create figure and axis with map projection
fig, ax = plt.subplots(1,1,subplot_kw={'projection':proj})
# draw some coastlines
ax.coastlines();


# plot SST for last month,
# i.e, the last time index, -1
sst["sst"][-1].plot(ax=ax,transform=data_proj,cbar_kwargs={"shrink": 0.7})
# plot extend of Nino3.4
nino34 = [lon1,lon2,lon2,lon1,lon1],[lat1,lat1,lat2,lat2,lat1]
ax.plot(*nino34,'k-',transform=data_proj)

In [ ]:
# create figure and axis with map projection
fig, ax = plt.subplots(1,1,subplot_kw={'projection':proj})
# draw some coastlines
ax.coastlines();


# plot SST anomalies for last month,
# i.e, the last time index, -1
sst["ssta"][-1].plot(ax=ax,transform=data_proj,cbar_kwargs={"shrink": 0.7})
# plot extend of Nino3.4
nino34 = [lon1,lon2,lon2,lon1,lon1],[lat1,lat1,lat2,lat2,lat1]
ax.plot(*nino34,'k-',transform=data_proj)

<b>Task 2: Add three other Nino regions to the map</b>

Hint:

nino12 =

ax.plot(*nino12,'k--',lw=1,transform=data_proj)



#### Section 1.2 - Calculate ONI in Nino3.4

In [ ]:
# slice dataset along Nino3.4 boundaries
# i.e., between "lon1" and "lon2",
# and between "lat1" and "lat2"

sst_a = sst["ssta"].loc[{
    'lat': slice(lat1,lat2),
    'lon': slice(lon1,lon2)
}]

# Mean over area, running mean over 3 months
sst_a_mean=sst_a.mean(dim=["lat","lon"])
sst_a_running=sst_a_mean.rolling(time=3, center=True).mean()

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))

timeticks=np.array([cftime.Datetime360Day(1900, 1, 1, 0, 0, 0, 0, has_year_zero=True),
                    cftime.Datetime360Day(1950, 1, 1, 0, 0, 0, 0, has_year_zero=True),
                    cftime.Datetime360Day(1958, 1, 1, 0, 0, 0, 0, has_year_zero=True),
                    cftime.Datetime360Day(2000, 1, 1, 0, 0, 0, 0, has_year_zero=True),],
                    dtype=object)

sst_a_running.plot.line(ax=ax, color='black',xticks=timeticks)

ax.set_xlim(cftime.Datetime360Day(1940, 1, 1, 0, 0, 0, 0, has_year_zero=True),cftime.Datetime360Day(2024, 1, 1, 0, 0, 0, 0, has_year_zero=True))

# Add lines to define ONI limits
ax.axhline(y=0.5, color='r', linestyle='--', label='Threshold')
ax.axhline(y=-0.5, color='b', linestyle='--', label='Threshold')

<b>Task 3: Which years are La Nina? El Nino ? Give 5 exmaples of each. </b>
    
Hint: Play with xlim and/or timeticks to find out dates
    


## Section 2: Climate in New Zealand


Next we look at ERA5 files for NZ and choose 2 locations: Auckland and Christchurch 

In [ ]:
# define a function that subtracts the mean along the "time" dimension
def subtract(x):
    return x - x.mean(dim="time")

# define our custom detrend function that also fills NaNs with 0s
def detrend_(x):
    return detrend(x.fillna(0),axis=0)

In [ ]:
lon_a=174.76
lat_a=-36.85

lon_c=172.64
lat_c=-43.53

era5=xr.open_dataset('era5_nz_1941_2023.nc')
era5

#rename valid_time into time
new_dims = {'valid_time': 'time'}
era5 = era5.rename(new_dims).set_coords([ 'time'])

In [ ]:
temp_a=era5.t2m.sel(longitude=lon_a,latitude=lat_a,method="nearest")

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
temp_a.plot.line(ax=ax, color='black')
ax.set_xlim(pd.to_datetime(['1940-01-01', '2024-01-01']))

In [ ]:
#Calculate temperature Anomaly (detrend+ remove mean value, remove seasonal signal)
# Save original variable:
temp0=era5

# apply the detrending, save variable after
era5 = era5.map(detrend_)
temp1=era5

# apply the subtraction of the monthly means, using "groupby", save the final variable
era5 = era5.groupby("time.month").map(subtract)
temp2=era5




In [ ]:
#Plot all three records to undertand what happened:
fig, axes = plt.subplots(3, 1, figsize=(8, 12))


temp0["t2m"].sel(longitude=lon_a,latitude=lat_a,method='nearest').plot(label="Original",lw=2,ax=axes[0])
axes[0].legend()
axes[0].set_xlim(pd.to_datetime(['1940-01-01', '1950-01-01']))


temp1["t2m"].sel(longitude=lon_a,latitude=lat_a,method='nearest').plot(label="detrended",lw=2,ax=axes[1])
axes[1].legend()
axes[1].set_xlim(pd.to_datetime(['1940-01-01', '1950-01-01']))


temp2["t2m"].sel(longitude=lon_a,latitude=lat_a,method='nearest').plot(label="anomaly",lw=2,ax=axes[2])
axes[2].legend()
axes[2].set_xlim(pd.to_datetime(['1940-01-01', '1950-01-01']))


<b> Task 4. plot the monthly means of temperature anomalies as time series for a location, for 1940-1950 </b>

Hint: add "resample(time="1MS").mean()" to convert the timeseries to the monthly mean

<b> Task 5: Repeat the plot with precipitaion records </b>

### Section 3. NZ and ENSO


https://niwa.co.nz/el-nino-and-la-nina

<b> Task 6: Read the page and aswer the questions: </b> 

1) Is there a uniform effect for the whole country?  

2) Is there a uniform effect for the whole year?



<b> Task 7: Plot ONI for the same time period as temperature and precipitation anomalies. </b>

#hint on time limit with ERSST time varibale format:

ax.set_xlim(cftime.Datetime360Day(1940, 1, 1, 0, 0, 0, 0, has_year_zero=True),cftime.Datetime360Day(1950, 1, 1, 0, 0, 0, 0, has_year_zero=True))


<b> Task 8: Describe the relationship between ONI, temperature and precipitaion records. Hint: focus on the two years with the strongest ONI signal.
    Use NIWA's page for guidance if needed. </b>


<b> Task 9: Plot a scatter plot of temperature and ONI relationship. To do so, convert era5 to monthly means again! </b>

Make a figure with 4 subplots. Each subplot must have a title and axes labels.

In [ ]:
#Define variables
a_tp=era5["tp"].resample(time="1MS").mean().sel(longitude=lon_a,latitude=lat_a,method='nearest').sel(time=slice("1941-01-01","1950-01-01"))
c_tp=era5["tp"].resample(time="1MS").mean().sel(longitude=lon_c,latitude=lat_c,method='nearest').sel(time=slice("1941-01-01","1950-01-01"))

a_temp=era5["t2m"].resample(time="1MS").mean().sel(longitude=lon_a,latitude=lat_a,method='nearest').sel(time=slice("1941-01-01","1950-01-01"))
c_temp=era5["t2m"].resample(time="1MS").mean().sel(longitude=lon_c,latitude=lat_c,method='nearest').sel(time=slice("1941-01-01","1950-01-01"))

oni=sst_a_running.sel(time=slice("1941-01-01","1950-01-01"))
np.shape(a), np.shape(b), np.shape(c)



In [ ]:
# Create a figure with 4 subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 12))


axes[0,0].scatter(a_tp, oni, .....

axes[0,1].

axes[1,0].

axes[1,1].


# Adjust the layout to avoid overlap
plt.tight_layout()


<b> Task 11: Repeat task 10 for 1940-2023  and discuss the change in results</b>

Add axis limits to make the plots easier to unsertand. 



In [ ]:
# Define variables


In [ ]:
# Create a figure with 2 subplots (1 row, 2 columns)


Bonus plot:  Will plotting the timeseries together help us explain what is happening?


In [ ]:
# plot the monthly means of anomalies as time series for a location 

timeticks2=np.array([cftime.Datetime360Day(1940, 1, 1, 0, 0, 0, 0, has_year_zero=True),
                     cftime.Datetime360Day(1950, 1, 1, 0, 0, 0, 0, has_year_zero=True),
                     cftime.Datetime360Day(1960, 1, 1, 0, 0, 0, 0, has_year_zero=True),
                     cftime.Datetime360Day(1970, 1, 1, 0, 0, 0, 0, has_year_zero=True), 
                     cftime.Datetime360Day(1980, 1, 1, 0, 0, 0, 0, has_year_zero=True),
                     cftime.Datetime360Day(1990, 1, 1, 0, 0, 0, 0, has_year_zero=True),
                     cftime.Datetime360Day(2000, 1, 1, 0, 0, 0, 0, has_year_zero=True),],
                     dtype=object)


#Plot all three records to undertand what happened:
fig, axes = plt.subplots(3, 1, figsize=(15, 15))


era5["t2m"].resample(time="1MS").mean() \
            .sel(longitude=lon_a,latitude=lat_a,method='nearest').plot(label="Auckland",lw=0.5,ax=axes[0],color='blue')
era5["t2m"].resample(time="1MS").mean() \
            .sel(longitude=lon_c,latitude=lat_c,method='nearest').plot(label="Christchurch",lw=0.5,ax=axes[0],color='red')
plt.legend();
axes[0].axhline(y=0, color='k', linestyle='--', label='Threshold')



era5["tp"].resample(time="1MS").mean() \
            .sel(longitude=lon_a,latitude=lat_a,method='nearest').plot(label="Auckland",lw=0.5,ax=axes[1],color='blue')
era5["tp"].resample(time="1MS").mean() \
            .sel(longitude=lon_c,latitude=lat_c,method='nearest').plot(label="Christchurch",lw=0.5,ax=axes[1],color='red')
plt.legend();
axes[1].axhline(y=0, color='k', linestyle='--', label='Threshold')




sst_a_running.plot.line(ax=axes[2],color='black',xticks=timeticks2)

axes[2].axhline(y=0.5, color='r', linestyle='--', label='Threshold')
axes[2].axhline(y=-0.5, color='b', linestyle='--', label='Threshold')


axes[0].set_xlim(pd.to_datetime(['1940-01-01', '2000-01-01']))
axes[1].set_xlim(pd.to_datetime(['1940-01-01', '2000-01-01']))
axes[2].set_xlim(cftime.Datetime360Day(1940, 1, 1, 0, 0, 0, 0, has_year_zero=True),cftime.Datetime360Day(2000, 1, 1, 0, 0, 0, 0, has_year_zero=True))